# Memory 

在许多应用场景中，我们希望维护一个“知识存储”，在特定步骤之前智能地将其中的事实加入智能体上下文中。
一个典型的例子就是 RAG 模式（Retrieval-Augmented Generation）——通过查询检索数据库中的相关信息，然后将其动态注入到智能体的上下文中。

AutoGen 提供了一个可扩展的 `Memory` 协议接口，来实现上述能力。它包括以下关键方法：

| 方法名              | 功能说明                                                 |
| ---------------- | ---------------------------------------------------------  |
| `add`            | 向记忆库中添加新条目                                         |
| `query`          | 从记忆库中检索与当前任务相关的信息                            |
| `update_context` | 修改智能体的 `model_context`，将检索到的信息动态加入（由 `AssistantAgent` 使用）|
| `clear`          | 清空记忆库中的所有内容                                     |
| `close`          | 清理记忆库使用的资源（如文件句柄、数据库连接等）            |

目前官方文档中已有的针对不同存储类型的Memory类：

1. ListMemory  

`ListMemory` 简单的基于列表的内存实现 Memory 功能，按时间顺序维护内存，并将最近的内存追加到模型的上下文中。该实现设计得简单明了、可预测，因此易于理解和调试。 

2. ChromaDBVectorMemory

`ChromaDBVectorMemory`使用由 ChromaDB 支持的矢量相似性搜索存储和检索内存。ChromaDBVectorMemory "提供了一种基于向量的内存实现，它使用 ChromaDB存储和检索基于语义相似性的内容。 它增强了代理在对话过程中调用上下文相关信息的能力。   

3. Mem0Memory

`Mem0Memory` 提供与 Mem0.ai 内存系统的集成。 它支持云端和本地后端，为代理提供高级内存功能。 该实现可处理适当的检索和上下文更新，因此适用于生产环境。

除此之外，我们还可以通过继承`Memory`基类以及重写其中的方法来实现我们想要存储记忆方式，例如，您可以实现一个使用向量数据库来存储和检索信息的自定义存储空间，或者一个使用机器学习模型来根据用户偏好等生成个性化响应的存储空间。我们将会以构建基于`Vector DBs`的存储方式的Memory系统为例对这一重要部分进行讲解。

另外，我们可基于Memory的功能下
✅ 应用场景示例：
- 🌐 RAG 查询文档数据库，支持问答或摘要任务

- 🤖 将过去对话片段或用户设定动态注入到当前轮上下文

- 📁 将用户上传的知识库（如 PDF、网页）作为记忆源动态接入



In [ ]:
import os
from dotenv import load_dotenv
from autogen_ext.models.openai import OpenAIChatCompletionClient

load_dotenv()
siliconflow_api_key = os.getenv("SILICONFLOW_API_KEY") # 读取你的 OPENAI API key

# 初始化 OpenAIChatCompletionClient 客户端，连接到硅基流动平台的 Qwen3-8B 模型
model_client = OpenAIChatCompletionClient(
    model="Qwen/Qwen3-14B",                # 指定要调用的模型名称，硅基流动平台上 Qwen 3-8B 模型
    base_url="https://api.siliconflow.cn/v1",  # 硅基流动平台的 API 访问地址
    api_key=siliconflow_api_key,  # 你的 API 密钥
    model_info={                        
        "family": "qwen",              
        "context_length": 8192,        
        "max_output_tokens": 2048,     
        "tool_choice_supported": True, 
        "tool_choice_required": False,  
        "structured_output": True,     
        "vision": False,                
        "function_calling": True,      
        "json_output": True,
        "multiple_system_messages":True
    },
)

我们要介绍autogen_core.memory下的两个类`ListMemoryConfig`和 `ListMemory`，方便之后的其他类型的Memory讲解

### ListMemoryConfig

`ListMemoryConfig`是 Autogen 框架中ListMemory组件的配置类，基于 Pydantic 的`BaseModel`构建，用于验证和序列化内存组件的配置参数。其核心作用是：

1. 定义ListMemory的初始化参数规范
2. 支持配置的序列化与反序列化（如 JSON 转换）
3. 提供类型提示和参数验证

关键属性说明：

`name`属性

类型：str | None

作用：为内存实例指定唯一名称，便于在多组件场景中识别

默认值：None（自动生成默认名称"default_list_memory"）

`memory_contents`属性

类型：List[MemoryContent]

作用：初始化时预加载的内存内容列表

默认值：通过default_factory=list生成空列表

注意：MemoryContent是 Autogen 的内存内容模型，包含content、metadata、mime_type等字段

我们来看官方文档里面的源代码会对其定义有更好的了解：

class ListMemoryConfig(BaseModel):
    """Configuration for ListMemory component."""

    name: str | None = None
    """Optional identifier for this memory instance."""
    memory_contents: List[MemoryContent] = Field(default_factory=list)
    """List of memory contents stored in this memory instance."""

### ListMemory

`ListMemory`是 Autogen 框架中基于列表的内存实现，用于存储和管理记忆内容，遵循Memory协议并集成Component配置系统。它以 chronological（时间顺序）方式存储内容，并支持将记忆内容整合到模型上下文中。

源代码中对`ListMemory`类的构造函数如下：

def __init__(self, name: str | None = None, memory_contents: List[MemoryContent] | None = None) -> None:
        self._name = name or "default_list_memory"
        self._contents: List[MemoryContent] = memory_contents if memory_contents is not None else []

其中第二个参数memory_contents就可以传入上面我们所讲的针对ListMemory的统一配置`ListMemoryConfig`，使得初始化的ListMemory实例都能获得ListMemoryConfig已经配备好的信息。

以下是一个带`ListMemory`记忆功能的智能体运用例子：

In [10]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core.memory import ListMemory, MemoryContent, MemoryMimeType
from autogen_ext.models.openai import OpenAIChatCompletionClient
# Initialize user memory
user_memory = ListMemory()

# Add user preferences to memory
await user_memory.add(MemoryContent(content="The weather should be in metric units", mime_type=MemoryMimeType.TEXT))

await user_memory.add(MemoryContent(content="Meal recipe must be vegan", mime_type=MemoryMimeType.TEXT))


async def get_weather(city: str, units: str = "imperial") -> str:
    if units == "imperial":
        return f"The weather in {city} is 73 °F and Sunny."
    elif units == "metric":
        return f"The weather in {city} is 23 °C and Sunny."
    else:
        return f"Sorry, I don't know the weather in {city}."


assistant_agent = AssistantAgent(
    name="assistant_agent",
    model_client=model_client,
    tools=[get_weather],
    memory=[user_memory],
)


In [17]:
# Run the agent with a task.
stream = assistant_agent.run_stream(task="What is the weather in New York?")
await Console(stream)

---------- TextMessage (user) ----------
What is the weather in New York?
---------- MemoryQueryEvent (assistant_agent) ----------
[MemoryContent(content='The weather should be in metric units', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='Meal recipe must be vegan', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)]
---------- ThoughtEvent (assistant_agent) ----------



---------- ToolCallRequestEvent (assistant_agent) ----------
[FunctionCall(id='0197c0adc898da07ad2e5e7026bad2dc', arguments=' {"city": "New York", "units": "metric"}', name='get_weather')]
---------- ToolCallExecutionEvent (assistant_agent) ----------
[FunctionExecutionResult(content='The weather in New York is 23 °C and Sunny.', name='get_weather', call_id='0197c0adc898da07ad2e5e7026bad2dc', is_error=False)]
---------- ToolCallSummaryMessage (assistant_agent) ----------
The weather in New York is 23 °C and Sunny.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What is the weather in New York?', type='TextMessage'), MemoryQueryEvent(source='assistant_agent', models_usage=None, metadata={}, content=[MemoryContent(content='The weather should be in metric units', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='Meal recipe must be vegan', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)], type='MemoryQueryEvent'), ThoughtEvent(source='assistant_agent', models_usage=None, metadata={}, content='\n\n', type='ThoughtEvent'), ToolCallRequestEvent(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=733, completion_tokens=118), metadata={}, content=[FunctionCall(id='0197c0adc898da07ad2e5e7026bad2dc', arguments=' {"city": "New York", "units": "metric"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant_agent', models_usage=None, metadata={}, content=[FunctionE

In [13]:
await assistant_agent._model_context.get_messages()

[UserMessage(content='What is the weather in New York?', source='user', type='UserMessage'),
 SystemMessage(content='\nRelevant memory content (in chronological order):\n1. The weather should be in metric units\n2. Meal recipe must be vegan\n', type='SystemMessage'),
 AssistantMessage(content=[FunctionCall(id='0197c09c432fe83c0acceea5353fc584', arguments=' {"city": "New York", "units": "metric"}', name='get_weather')], thought='\n\n', source='assistant_agent', type='AssistantMessage'),
 FunctionExecutionResultMessage(content=[FunctionExecutionResult(content='The weather in New York is 23 °C and Sunny.', name='get_weather', call_id='0197c09c432fe83c0acceea5353fc584', is_error=False)], type='FunctionExecutionResultMessage'),
 UserMessage(content='What is the weather in New York?', source='user', type='UserMessage'),
 SystemMessage(content='\nRelevant memory content (in chronological order):\n1. The weather should be in metric units\n2. Meal recipe must be vegan\n', type='SystemMessage'),

我们可以观察到，`assistant_agent` 的 `model_context` 实际上已经被检索到的记忆条目更新了。`transform` 方法用于将这些记忆条目格式化为一个代理可以使用的字符串。在这种情况下，我们只是将每个记忆条目的内容简单地连接成一个字符串。

我们可以看到，上述返回的天气信息是以摄氏度为单位的，正如用户偏好中所设定的那样。

同样地，假设我们提出一个关于生成饮食计划的问题，代理也能够从记忆存储中检索到相关信息，并提供一个个性化的（素食）回应。

In [ ]:
stream = assistant_agent.run_stream(task="Write brief meal recipe with broth")
await Console(stream)

讲完了ListMemory的基本用法，我们简单举一下 `ChromaDBVectorMemory` 和 `Mem0Memory` 的例子，他们的用法大同小异。

### ChromaDBVectorMemory

`ChromaDBVectorMemory`将我们传入的知识库进行向量化后存储在ChromaDB中，所以要传入你想要的Embedding方法，目前官方源代码中定义了以下几个EmbeddingFunctionConfig

1. SentenceTransformerEmbeddingFunctionConfig

2. OpenAIEmbeddingFunctionConfig

3. DefaultEmbeddingFunctionConfig

4. CustomEmbeddingFunctionConfig

他们的区别是所使用的Embedding模型不一样。

建议用户直接前往autogen_ext/memory/chromadb/_chroma_configs.py(V0.6.2)中查看源代码中的定义，这样你会有更清晰的理解。

### ChromaDBVectorMemory

In [ ]:
import tempfile
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core.memory import MemoryContent, MemoryMimeType
from autogen_ext.memory import chromadb
# (
#     ChromaDBVectorMemory,
#     PersistentChromaDBVectorMemoryConfig,
#     SentenceTransformerEmbeddingFunctionConfig,
# )
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Use a temporary directory for ChromaDB persistence
with tempfile.TemporaryDirectory() as tmpdir:
    chroma_user_memory = ChromaDBVectorMemory(
        config=PersistentChromaDBVectorMemoryConfig(
            collection_name="preferences",
            persistence_path=tmpdir,  # Use the temp directory here
            k=2,  # Return top k results
            score_threshold=0.4,  # Minimum similarity score
            embedding_function_config=SentenceTransformerEmbeddingFunctionConfig(
                model_name="all-MiniLM-L6-v2"  # Use default model for testing
            ),
        )
    )
    # Add user preferences to memory
    await chroma_user_memory.add(
        MemoryContent(
            content="The weather should be in metric units",
            mime_type=MemoryMimeType.TEXT,
            metadata={"category": "preferences", "type": "units"},
        )
    )

    await chroma_user_memory.add(
        MemoryContent(
            content="Meal recipe must be vegan",
            mime_type=MemoryMimeType.TEXT,
            metadata={"category": "preferences", "type": "dietary"},
        )
    )

    model_client = OpenAIChatCompletionClient(
        model="gpt-4o",
    )

    # Create assistant agent with ChromaDB memory
    assistant_agent = AssistantAgent(
        name="assistant_agent",
        model_client=model_client,
        tools=[get_weather],
        memory=[chroma_user_memory],
    )

    stream = assistant_agent.run_stream(task="What is the weather in New York?")
    await Console(stream)

    # await model_client.close()
    # await chroma_user_memory.close()


### Mem0Memory

注意：Mem0Memory在0.6.2及之后的版本才有的

In [ ]:
import io
import logging
import uuid
from contextlib import redirect_stderr, redirect_stdout
from datetime import datetime
from typing import Any, Dict, List, Optional, TypedDict, cast

from autogen_core import CancellationToken, Component, ComponentBase
from autogen_core.memory import Memory, MemoryContent, MemoryQueryResult, UpdateContextResult
from autogen_core.model_context import ChatCompletionContext
from autogen_core.models import SystemMessage
from pydantic import BaseModel, Field
from typing_extensions import Self

try:
    from mem0 import Memory as Memory0
    from mem0 import MemoryClient
except ImportError as e:
    raise ImportError("`mem0ai` not installed. Please install it with `pip install mem0ai`") from e

logger = logging.getLogger(__name__)
logging.getLogger("chromadb").setLevel(logging.ERROR)
 # Create a cloud Mem0Memory
memory = Mem0Memory(is_cloud=True)
# Add something to memory
await memory.add(MemoryContent(content="Important information to remember"))
# Retrieve memories with a search query
results = await memory.query("relevant information")

## 接下来，我们将介绍`Memory`的复杂应用

基于上述介绍的Memory，我们现在可以构建一个简单的`RAG Agent`。

在构建AI系统中常见的RAG（检索增强生成）模式包含两个不同的阶段：
 
1. 索引：加载文档，将它们分块，并将它们存储在矢量数据库中

2. 检索：在会话运行时查找和使用相关块

在前面的示例中，我们手动将项添加到内存中并将它们传递给代理。在实践中，索引过程通常是自动化的，并且基于更大的文档源，如产品文档、内部文件或知识库。

 首先，让我们创建一个简单的`文档索引器`，我们将使用它来加载文档、对它们进行分组并将它们存储在内存存储区中。


In [4]:
import re
from typing import List
import aiofiles
import aiohttp
from autogen_core.memory import Memory, MemoryContent, MemoryMimeType


class SimpleDocumentIndexer:
    """Basic document indexer for AutoGen Memory."""

    def __init__(self, memory: Memory, chunk_size: int = 1500) -> None:
        self.memory = memory
        self.chunk_size = chunk_size

    async def _fetch_content(self, source: str) -> str:
        """Fetch content from URL or file."""
        if source.startswith(("http://", "https://")):
            async with aiohttp.ClientSession() as session:
                async with session.get(source) as response:
                    return await response.text()
        else:
            async with aiofiles.open(source, "r", encoding="utf-8") as f:
                return await f.read()

    def _strip_html(self, text: str) -> str:
        """Remove HTML tags and normalize whitespace."""
        text = re.sub(r"<[^>]*>", " ", text)
        text = re.sub(r"\s+", " ", text)
        return text.strip()

    def _split_text(self, text: str) -> List[str]:
        """Split text into fixed-size chunks."""
        chunks: list[str] = []
        # Just split text into fixed-size chunks
        for i in range(0, len(text), self.chunk_size):
            chunk = text[i : i + self.chunk_size]
            chunks.append(chunk.strip())
        return chunks
        
    async def index_documents(self, sources: List[str]) -> int:
        """Index documents into memory."""
        total_chunks = 0
        for source in sources:
            try:
                content = await self._fetch_content(source)
                # Strip HTML if content appears to be HTML
                if "<" in content and ">" in content:
                    content = self._strip_html(content)
                chunks = self._split_text(content)
                for i, chunk in enumerate(chunks):
                    await self.memory.add(
                        MemoryContent(
                            content=chunk, mime_type=MemoryMimeType.TEXT, metadata={"source": source, "chunk_index": i}
                        )
                    )

                total_chunks += len(chunks)
            except Exception as e:
                print(f"Error indexing {source}: {str(e)}")

        return total_chunks

现在让我们使用我们的索引器`ChromaDBVectorMemory`来构建一个完整的RAG Agent：

In [ ]:
import os
from pathlib import Path

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.memory import chromadb
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Initialize vector memory

rag_memory = ChromaDBVectorMemory(
    config=PersistentChromaDBVectorMemoryConfig(
        collection_name="autogen_docs",
        persistence_path=os.path.join(str(Path.home()), ".chromadb_autogen"),  #设置存储的硬盘区域
        k=3,  # Return top 3 results
        score_threshold=0.4,  # Minimum similarity score
    )
)

await rag_memory.clear()  # Clear existing memory


# Index AutoGen documentation
async def index_autogen_docs() -> None:
    indexer = SimpleDocumentIndexer(memory=rag_memory)
    sources = [
        "https://raw.githubusercontent.com/microsoft/autogen/main/README.md"
    ]
    chunks: int = await indexer.index_documents(sources)
    print(f"Indexed {chunks} chunks from {len(sources)} AutoGen documents")


await index_autogen_docs()


上述操作后，我们已经成功把我们传入的文档存储在ChromaDBVector中，现在我们准备讲改Memory集成到智能体之中。

In [ ]:
# Create our RAG assistant agent
rag_assistant = AssistantAgent(
    name="rag_assistant", model_client=model_client, memory=[rag_memory]
)

# Ask questions about AutoGen
stream = rag_assistant.run_stream(task="What is AgentChat?")
await Console(stream)

# Remember to close the memory when done
await rag_memory.close()


这个实现提供了一个RAG代理，可以根据AutoGen文档回答问题。当提出问题时，Memory系统检索相关的块并将它们添加到上下文中，使助手能够生成知情的回答。